In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv
/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv
/kaggle/input/llm-detect-ai-generated-text/test_essays.csv
/kaggle/input/llm-detect-ai-generated-text/train_essays.csv


In [2]:
#os.mkdir('/kaggle/checkpoints')

In [3]:
train_essays = pd.read_csv(r'/kaggle/input/llm-detect-ai-generated-text/train_essays.csv')
train_prompts = pd.read_csv(r'/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv')
test_essays = pd.read_csv(r'/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')

In [4]:
train_prompts.iloc[0].instructions

'Write an explanatory essay to inform fellow citizens about the advantages of limiting car usage. Your essay must be based on ideas and information that can be found in the passage set. Manage your time carefully so that you can read the passages; plan your response; write your response; and revise and edit your response. Be sure to use evidence from multiple sources; and avoid overly relying on one source. Your response should be in the form of a multiparagraph essay. Write your essay in the space provided.'

In [5]:
train_prompts.iloc[0].source_text[:30]

'# In German Suburb, Life Goes '

In [6]:
train_prompts.columns

Index(['prompt_id', 'prompt_name', 'instructions', 'source_text'], dtype='object')

In [7]:
train_essays.head()

,id,prompt_id,text,generated
0,0059830c,0,Cars. Cars have been around since they became ...,0
1,005db917,0,Transportation is a large necessity in most co...,0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0
3,00940276,0,How often do you ride in a car? Do you drive a...,0
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0


In [8]:
# 0.41.3.post2
# accelerate==0.25.0
#transformers4.35.2
!pip install transformers==4.35.2
!pip install accelerate==0.25.0
!pip install bitsandbytes==0.41.3.post2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 78.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.33.0
    Uninstalling transformers-4.33.0:
      Successfully uninstalled transformers-4.33.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 1.8 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.22.0
    Uninstalling accelerate-0.22.0:
      Successfully uninstalled accelerate-0.22.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 13.0 MB/s eta 0:00:00


In [9]:
%pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.3/330.3 kB 18.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.16.4
    Uninstalling huggingface-hub-0.16.4:
      Successfully uninstalled huggingface-hub-0.16.4
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
Note: you may need to restart the kernel to use updated packages.


In [10]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import json
import re

In [11]:
def load_quantized_model(model_name: str):
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        load_in_4bit=True,
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config
    )

    return model

def initialize_tokenizer(model_name: str):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.bos_token_id = 1  # Set beginning of sentence token id
    return tokenizer


In [12]:
model_name = 'mistralai/Mistral-7B-Instruct-v0.2'


model = load_quantized_model(model_name)

tokenizer = initialize_tokenizer(model_name)

# Define stop token ids
stop_token_ids = [0]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [13]:
def make_prompt(
        example_indices_full,
        example_index_to_classify,
        val_essays=train_essays
):
    prompt = '''[INST] Dectect whether the Essay is generated by a language \ 
    a large langauge model (LLM) or written by human.
Return the answer as JSON format STRICTLY as per OUTPUT_FORMAT.
Please note:
  - Human style of writing is different than that of langauge models as Human may make  \
  grammatical and/or spelling mistakes, whearas LLM may generate nonsensical text.
  - Your response should always start with ```json  and end with ``` quotes.
  - Always return an answer with a score between 0 and 1 where one is the highest./
     This score indicate how confident you are of your answer.

### OUTPUT_FORMAT:

```json
{
  "generated_by" : <Human or LLM>,
  "score": <between 0 and 1>
}
```

## Examples:

'''
    for index in example_indices_full:
        text = train_essays.iloc[index].text
        is_generated_by = "Human" if train_essays.iloc[index].generated == 0 else "LLM"
        output = {
            #"essay": text, # removed as the text was taking too much
            "generated_by" : is_generated_by,
            "score": 0.9
        }
        output_str = json.dumps(output)

        prompt += f"""
### Essay:

{text}

### OUTPUT:
```json
{output_str}

```
"""

    text = val_essays.iloc[example_index_to_classify].text
    #is_generated_by = "Human" if essays.iloc[example_index_to_classify].generated == 0 else "LLM"
    prompt += f"""

### Essay:

{text}

### OUTPUT:
[/INST]
"""
    return prompt


In [14]:
# mistral

In [15]:
example_indices_full = [40, 740]# , 740 # llm, the prompt is very long
example_index_to_classify = 200#704

few_shot_prompt = make_prompt(
    example_indices_full=example_indices_full,
    example_index_to_classify=example_index_to_classify,
    val_essays=train_essays
)

print(few_shot_prompt)

[INST] Dectect whether the Essay is generated by a language \ 
    a large langauge model (LLM) or written by human.
Return the answer as JSON format STRICTLY as per OUTPUT_FORMAT.
Please note:
  - Human style of writing is different than that of langauge models as Human may make    grammatical and/or spelling mistakes, whearas LLM may generate nonsensical text.
  - Your response should always start with ```json  and end with ``` quotes.
  - Always return an answer with a score between 0 and 1 where one is the highest./
     This score indicate how confident you are of your answer.

### OUTPUT_FORMAT:

```json
{
  "generated_by" : <Human or LLM>,
  "score": <between 0 and 1>
}
```

## Examples:


### Essay:

By limiting car usage you would be making it a healther environment, and a safer one!

there would be less pollution in the air and that would make the air better for us to breath.

Sure by limiting the usage of cars would have an effect on people its also doing us a favor wether w

In [16]:
def get_response(user_message):
    encoded = tokenizer(user_message, return_tensors="pt")
    if torch.cuda.is_available():  # Ensure input tensors are on the GPU if model is on GPU
        model_inputs = encoded.to('cuda')
    response = model.generate(**model_inputs, max_new_tokens=4096, do_sample=False)
    return response

In [17]:
%timeit
outputs = get_response(few_shot_prompt)
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(decoded_output)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Dectect whether the Essay is generated by a language \ 
    a large langauge model (LLM) or written by human.
Return the answer as JSON format STRICTLY as per OUTPUT_FORMAT.
Please note:
  - Human style of writing is different than that of langauge models as Human may make    grammatical and/or spelling mistakes, whearas LLM may generate nonsensical text.
  - Your response should always start with ```json  and end with ``` quotes.
  - Always return an answer with a score between 0 and 1 where one is the highest./
     This score indicate how confident you are of your answer.

### OUTPUT_FORMAT:

```json
{
  "generated_by" : <Human or LLM>,
  "score": <between 0 and 1>
}
```

## Examples:


### Essay:

By limiting car usage you would be making it a healther environment, and a safer one!

there would be less pollution in the air and that would make the air better for us to breath.

Sure by limiting the usage of cars would have an effect on people its also doing us a favor wether w

In [18]:
import time
# now apply to test set to see the reustll


tic = time.time()
results = []
for i, row in train_essays.iterrows():
    example_indices_full = [40, 740]# , 740 # llm, the prompt is very long
    if i in example_indices_full:
        continue
    example_index_to_classify = i# 200#704
    #print(row)

    few_shot_prompt = make_prompt(
        example_indices_full=example_indices_full,
        example_index_to_classify=example_index_to_classify,
        val_essays=train_essays
    )

    #print(few_shot_prompt)

    outputs = get_response(few_shot_prompt)
    start_time = time.time()
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    model_gen_time = time.time() - start_time
    #print(decoded_output)

        # Search for the pattern in the text
    match = pattern.search(decoded_output)

    if match:
        json_content = json.loads(match.group(1).strip())
        print(json_content, type(json_content))
    else:
        print("Pattern not found.")
    llm_out = json_content.get("generated_by", None)
    if llm_out is not None:
        predicted = 0 if "Human" else 1

    # test does not have generated, so None is just placeholder
    gt_class = None
    gt_class = train_essays.iloc[i].generated
    results.append({
        "id": i,
        "llm-otuput": llm_out,
        "human-output": gt_class,
        "time": model_gen_time
    })
    print("#"*50)
toc = time.time()
toe = toc - tic
print(toe)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


NameError: name 'pattern' is not defined